#### Magic Formula por Joel Greenblatt

O programa executa a Fórmula Magica estabelecida por Joel GreenBlatt

As etapas que foram utilizadas:
- Download dos dados das ações das empresas sem filtro prévio em formato .csv
- Determinar o valor de liquidez média diária para iniciar os filtros
- De forma ascendente, ordenar os dados utilizando a coluna EV/EBIT e inserir números de 1 ao número de linhas da planilha em uma coluna "RANKING EV/EBIT"
- De forma descendente, ordenar os dados utilizando a coluna ROIC e inserir números, de 1 ao número de linhas da planilha em uma coluna "RANKING ROIC"
- Criar uma coluna "SOMA RANKING" e somar as colunas "RANKING EV/EBIT" e "RANKING ROIC"
- De forma ascendente, ordenar os dados utilizando a coluna "SOMA RANKING"
- Exibir as 20 empresas dessa lista.
- Fazer uma análise mais detalhada dessas empresas, pois pode haver divergências nos balanços

In [1]:
import pandas as pd
import locale
import os

locale.setlocale(locale.LC_ALL, '')

folder = os.getcwd() # Pasta de trabalho onde está salvo este Jupiter notebook

# Inserindo o mês, onde:
# Janeiro: 01, Fevereiro: 02, etc
lista_mes = ['01','02','03','04','05','06','07','08','09','10','11','12']

while True:
    try:
        mes = input('Insira o mes: ') 
        if (mes not in lista_mes):
            raise ValueError('Mes invalido')
    except ValueError:
        print('Mês inválido')
    else:
        break
        
file = '\\acoes_' + mes + '.csv' # Nome do arquivo que está salvo na pasta de trabalho onde está rodando o notebook

Insira o mes: 09


In [2]:
lista_arquivos_pasta = []

for diretorio, subpastas, arquivos in os.walk(folder):
    for arquivo in arquivos:
        if (arquivo.endswith('.csv') & arquivo.startswith('acoes_')):
            lista_arquivos_pasta.append(arquivo)

In [3]:
# Lendo o arquivo CSV
df_acoes = pd.read_csv(folder+file, engine = 'python', sep=';')

In [4]:
# Converte os valores que estão como string para tipo float
def converte_formato(df, coluna):
    df[coluna] = df[coluna].apply(lambda x:locale.atof(x.split()[-1]))
    return df[coluna]

In [5]:
# Convertendo as colunas EV/EBIT, ROIC, Liquidez diária e Preço para o tipo FLOAT
df_acoes_copy = df_acoes.copy().dropna(subset=['ROIC', 'EV/EBIT','PRECO',' LIQUIDEZ MEDIA DIARIA'])

df_acoes_copy['EV/EBIT'] = converte_formato(df_acoes_copy, 'EV/EBIT')
df_acoes_copy['ROIC'] = converte_formato(df_acoes_copy, 'ROIC')
df_acoes_copy['PRECO'] = converte_formato(df_acoes_copy, 'PRECO')
df_acoes_copy[' LIQUIDEZ MEDIA DIARIA'] = converte_formato(df_acoes_copy, ' LIQUIDEZ MEDIA DIARIA')

In [6]:
# Etapas:
# 1) Pegar o nome do papel
# 2) Identificar as empresas que tem vários tipos de papéis (Ex: TAEE4 e TAEE3)
# 3) Comparar a liquidez diária e manter, apenas, o papel onde a liquidez diaria é a maior de todas

In [8]:
# Determinando a liquidez diária para o filtro
while True:
    try:
        input_liq_diaria = float(input('Liquidez diaria: '))
        if input_liq_diaria <= 0:
            raise ValueError("Valor fora do range")
    except ValueError:
        print('Valor inválido')
    else:
        break

Liquidez diaria: -1
Valor inválido
Liquidez diaria: afsdfasdf
Valor inválido
Liquidez diaria: asdfad
Valor inválido
Liquidez diaria: 300000


In [10]:
# Ordenando pela Liquidez media diária
df_acoes_copy = df_acoes_copy.loc[(df_acoes_copy[' LIQUIDEZ MEDIA DIARIA'] >= input_liq_diaria) & 
                                  (df_acoes_copy['EV/EBIT'] > 0) & 
                                  (df_acoes_copy['ROIC'] > 0) ]\
                                .sort_values(by=' LIQUIDEZ MEDIA DIARIA', ascending=False)

In [11]:
# Inserindo uma coluna 'TICKER_SEM_NUM' para remover os papéis da mesma empresa.
# Primeiro, ordenamos as coluna tomando como base a coluna ' LIQUIDEZ DIARIA MEDIA'.
# Apagamos a coluna 'TICKER_SEM_NUM' pois não será mais utilizada

df_acoes_copy['TICKER_SEM_NUM'] = df_acoes_copy['TICKER'].apply(lambda y: y[:-1])
df_acoes_copy = df_acoes_copy.drop_duplicates(subset=['TICKER_SEM_NUM']).drop(columns='TICKER_SEM_NUM')

In [12]:
# RANKING EV/EBIT
lista_acoes = list(range(1, len(df_acoes_copy)+1,1)) # Cria uma lista com o número de linhas do df

df_acoes_copy = df_acoes_copy.sort_values(by='EV/EBIT')
df_acoes_copy['RANKING EV/EBIT'] = lista_acoes

In [13]:
# RANKING ROIC
df_acoes_copy = df_acoes_copy.sort_values(by='ROIC', ascending=False)
df_acoes_copy['RANKING ROIC'] = lista_acoes

In [14]:
# Somando as colunas RANKING EV/EBIT e RANKING ROIC para criar o ranking das ações
df_acoes_copy['SOMA RANKING'] = df_acoes_copy['RANKING EV/EBIT'] + df_acoes_copy['RANKING ROIC']

In [15]:
# Ordenando o df pela coluna SOMA RANKING de forma crescente
df_acoes_copy = df_acoes_copy.sort_values(by='SOMA RANKING').reset_index()
df_acoes_copy[['TICKER','PRECO','SOMA RANKING']].head(20) # Exibindo as 20 empresa de acordo com o ranking da FM

,TICKER,PRECO,SOMA RANKING
0,BRAP4,23.14,5
1,FHER3,16.17,7
2,G2DI33,3.17,7
3,SYNE3,4.81,11
4,UNIP6,90.24,17
5,TASA4,19.00,19
6,GOAU4,10.27,20
7,ETER3,12.67,21
8,GGBR4,24.63,24
9,CMIN3,3.96,25


In [16]:
# Criando um arquivo Excel indicando o mês no final do arquivo
df_acoes_copy.head(20).to_excel('Lista_AcoesFM_'+ mes + '.xlsx', columns=['TICKER', 'PRECO', 'SOMA RANKING'])